Validate JAVA runtime

In [2]:
import os

# Set Java environment variables
java_home = '/Users/tasbihaasim/.sdkman/candidates/java/current'
os.environ['JAVA_HOME'] = java_home
os.environ['PATH'] = f"{java_home}/bin:{os.environ['PATH']}"

# Check if JAVA_HOME is set correctly
print("JAVA_HOME:", os.environ.get('JAVA_HOME'))

# Check if the correct java binary is in PATH
os.system('java -version')


JAVA_HOME: /Users/tasbihaasim/.sdkman/candidates/java/current


openjdk version "17" 2021-09-14
OpenJDK Runtime Environment Temurin-17+35 (build 17+35)
OpenJDK 64-Bit Server VM Temurin-17+35 (build 17+35, mixed mode)


0

Import Relevant Packages

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,BooleanType,DoubleType
from pyspark.sql.functions import col, weekofyear, year
from pyspark.sql.functions import to_timestamp, date_format, concat_ws, col, to_date



Parse Mode

In [4]:

def parse(session):
    dataframe = session.read.json("dataset.json")
    # Print the original schema
    dataframe.printSchema()
    
    # Convert the 'timestamp' column to timestamp type and add it as a new column

    
    dataframe = dataframe.withColumn("new_timestamp", to_date("timestamp", "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))
    
    # Print the schema after the conversion
    dataframe.printSchema()
    # cast the timstampt column to be of a type timestamp
    # dataframe.show()
    app_loaded_events = dataframe.filter(dataframe.event == "app_loaded")
    registered_events = dataframe.filter(dataframe.event == "registered")
    app_loaded_events.write.mode("overwrite").parquet("user/events/app_loaded/")
    registered_events.write.mode("overwrite").parquet("user/events/registered/")

Statistics mode

In [6]:
def statistics(session):
    # Load the parquet files into DataFrames
    app_loaded_df = session.read.parquet("user/events/app_loaded/")
    registered_df = session.read.parquet("user/events/registered/")

    # Extract year and week from the timestamp
    app_loaded_df = app_loaded_df.withColumn("year", year("new_timestamp")).withColumn("week_of_year", weekofyear("new_timestamp"))
    registered_df = registered_df.withColumn("year", year("new_timestamp")).withColumn("week_of_year", weekofyear("new_timestamp"))

    # Create temporary views for SQL operations
    app_loaded_df.createOrReplaceTempView("apploaded")
    registered_df.createOrReplaceTempView("registered")
    
    # # Run the first query to get the last app load time for each user
    # lastload = '''
    # SELECT initiator_id, MAX(new_timestamp) AS last_app_load_time
    # FROM apploaded
    # GROUP BY initiator_id
    # '''

    # lastload_df = session.sql(lastload)
    # lastload_df.createOrReplaceTempView("lastload_df")  # Using 'lastload_df' as the view name

    # total_non_distinct_appload_events = '''
    # SELECT 
    # initiator_id, 
    # COUNT(*) AS load_count
    # FROM 
    #     apploaded
    # GROUP BY 
    #     initiator_id
    # HAVING 
    #     COUNT(*) > 1;
    # '''
    # print("users who loaded the app more than once")
    # countt = session.sql(total_non_distinct_appload_events)
    # countt.createOrReplaceTempView("countt")
    # countt.show()

    # # Debugging: Check the content of the 'lastload_df' view
    # print("Last Load DF:")
    # lastload_df.show()

    # query = '''
    # SELECT r.initiator_id
    # FROM registered r
    # JOIN apploaded l
    # ON r.initiator_id = l.initiator_id
    # WHERE 
    #     (r.year = year(l.last_app_load_time) AND r.week_of_year = weekofyear(l.last_app_load_time) - 1)
    #     OR 
    #     ((r.year = year(l.last_app_load_time) + 1) AND (weekofyear(l.last_app_load_time) = 52 AND r.week_of_year = 1))
    # '''
    query = """
    SELECT DISTINCT r.initiator_id
    FROM registered r
    JOIN apploaded a
    ON r.initiator_id = a.initiator_id
    AND (r.year = a.year AND r.week_of_year = a.week_of_year-1)

    """
    dfsql = session.sql(query)  # This should refer to 'lastload_df'

    # Debugging: Check the result of the query
    print("Filtered DF:")
    dfsql.show()

    # Run the query to get all users
    query2 = '''
    SELECT DISTINCT initiator_id FROM registered
    '''
    dfsql_all_users = session.sql(query2)

    # Calculate the statistics
    registered_users = dfsql.count()  # users that registered themselves the week after loading the app
    total_users_count = dfsql_all_users.count()  # total users

    percentage = (registered_users / total_users_count) * 100

    return percentage


In [7]:

session = SparkSession.builder.appName("first pyspark application")\
.master("local[*]")\
.getOrCreate()

print(session.sparkContext.appName)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/12 15:37:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


first pyspark application


In [8]:
parse(session)

root
 |-- browser_version: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- device_type: string (nullable = true)
 |-- event: string (nullable = true)
 |-- initiator_id: long (nullable = true)
 |-- timestamp: string (nullable = true)

root
 |-- browser_version: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- device_type: string (nullable = true)
 |-- event: string (nullable = true)
 |-- initiator_id: long (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- new_timestamp: date (nullable = true)



24/08/12 15:38:10 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/08/12 15:38:10 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
24/08/12 15:38:10 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
24/08/12 15:38:10 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
24/08/12 15:38:10 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
24/08/12 15:38:11 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
24/08/12 15:38:11 WARN MemoryManager: Total allocation exceeds 95,0

In [9]:
print(statistics(session))

Filtered DF:
+-------------------+
|       initiator_id|
+-------------------+
|3074457347138120171|
|3074457347155748055|
|3074457347157300243|
|3074457347124748055|
|3074457347143760172|
|3074457347146488765|
|3074457347153067794|
|3074457347173997067|
|3074457347139582185|
|3074457347131568759|
|3074457347132133806|
|3074457347129021537|
|3074457347138829175|
|3074457347138816721|
|3074457347172960046|
|3074457347150067564|
|3074457347157776707|
|3074457347131400948|
|3074457347138182666|
|3074457347146917642|
+-------------------+
only showing top 20 rows

7.571409047423808


In [434]:
# app_loaded_df = session.read.parquet("user/events/app_loaded/")
# registered_df = session.read.parquet("user/events/registered/")
session.read.parquet("user/events/app_loaded/").show()

+---------------+--------+-------+-----------+----------+-------------------+--------------------+-------------+
|browser_version|campaign|channel|device_type|     event|       initiator_id|           timestamp|new_timestamp|
+---------------+--------+-------+-----------+----------+-------------------+--------------------+-------------+
|               |    NULL|   NULL| mobile-app|app_loaded|3074457346125985645|2020-01-07T18:18:...|   2020-07-01|
|           65.0|    NULL|   NULL|    desktop|app_loaded|3074457347026799765|2020-01-07T18:18:...|   2020-07-01|
|           56.0|    NULL|   NULL|desktop-app|app_loaded|3074457347134362674|2020-01-07T18:19:...|   2020-07-01|
|           79.0|    NULL|   NULL|     mobile|app_loaded|3074457347134440587|2020-01-07T18:19:...|   2020-07-01|
|           79.0|    NULL|   NULL|    desktop|app_loaded|3074457346043002471|2020-01-07T18:19:...|   2020-07-01|
|           79.0|    NULL|   NULL|    desktop|app_loaded|3074457347034158585|2020-01-07T18:20:..